# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [660]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [661]:
from copy import copy
import math

# adjacency list implementation of a non-directed weighted graph
class Network:
    def __init__(self):
        # stations are stored in a dictionary where each key is a station name and
        # its value is a list of station connections
        self.graph = {}

    def add_station(self, station):
        self.graph[station.name] = [station] # the first station's connection list is the station object itself

    def add_edge(self, start_station_name, end_station_name, weight=None):
        if weight is None:
            edge_weight = self.distance(start_station_name, end_station_name)
        else:
            edge_weight = weight
        station1 = self.get_station(start_station_name)
        station2 = self.get_station(end_station_name)
        station1.set_weight(edge_weight)
        station2.set_weight(edge_weight)
        self.graph[start_station_name].append(station2)
        self.graph[end_station_name].append(station1)

    def get_station(self, station_name):
        station = self.graph[station_name][0]
        return copy(station)

    def get_station_connections(self, station_name):
        return self.graph[station_name][1:]

    def get_station_names(self):
        return list(self.graph.keys())

    def exists_edge(self, station_name1, station_name2):
        for station_connection in self.get_station_connections(station_name1):
            if station_connection.name == station_name2:
                return True
        return False

    def is_connected(self):
        visited = {station_name: False for station_name in self.get_station_names()} # stores whether each station has been visited
        visited[self.get_station_names()[0]] = True # start at the fromS
        queue = deque()
        queue.appendleft(self.get_station_names()[0])

        while queue: # while there are still unvisited stations
            current_station_name = queue.pop()
            for station_connection in self.get_station_connections(current_station_name):
                # for each unvisited neighbour of the current station,
                # increment its distance from the source node by one and add it to the queue
                if not visited[station_connection.name]:
                    visited[station_connection.name] = True
                    queue.appendleft(station_connection.name)

        for val in list(visited.values()):
            if not val: return False
        return True

    def size(self):
        return len(self.graph)

    # computes the haversine distance between two stations
    # https://stackoverflow.com/a/4913653/4934861
    def distance(self, station1_name, station2_name):
        lon1, lat1, lon2, lat2 = map(math.radians, [self.get_station(station1_name).longitude,
                                               self.get_station(station1_name).latitude,
                                               self.get_station(station2_name).longitude,
                                               self.get_station(station2_name).latitude])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.asin(math.sqrt(a))
        r = 3956  # radius of earth in miles.
        return c * r

    # lazy implementation of Prim's algorithm to get the minimum spanning tree (mst) from the network
    def get_mst(self):

        def visit(station_name):
            reached_set.add(station_name)
            # this for loop adds all unconnected neighbours of the mst to the unreached min-heap
            for station_connection in self.get_station_connections(station_name):
                if not station_connection.name in reached_set:
                    mst_edge = MSTEdge(station_name, station_connection.name, station_connection.get_weight())
                    unreached_set.insert(mst_edge)

        reached_set = set()
        mst = []
        unreached_set = MinHeap() # initialise a min-heap to keep track of the stations yet to be visited connected to the current mst
        visit(self.get_station_names()[0]) # start at an arbitary station

        while not unreached_set.is_empty() and len(mst) < self.size()-1:
            e = unreached_set.get_min() # remove the lowest weighted unvisited edge connected to the current mst
            v = e.get_start_station_name()
            w = e.get_end_station_name()
            if v in reached_set and w in reached_set: continue # edge already in mst
            mst.append(e) # add edge to mst
            if not v in reached_set: visit(v)
            if not w in reached_set: visit(w)

        # this block of code generates and returns a Network object from the mst list
        mst_network = Network()
        for station_name in self.get_station_names():
            mst_network.add_station(self.get_station(station_name))
        for edge in mst:
            mst_network.add_edge(edge.get_start_station_name(), edge.get_end_station_name(), edge.get_weight())

        return mst_network

    # returns an eulerian tour of the graph using Hierholzer’s Algorithm
    # https://en.wikipedia.org/wiki/Eulerian_path
    # https://www-m9.ma.tum.de/graph-algorithms/hierholzer/index_en.html
    def get_eulerian_tour(self):
        graph = copy(self.graph) # graph is copied so popping stations' neighbours doesn't affect the object's graph attribute
        current_path = [self.get_station_names()[0]] # initialise current path stack with an arbitrary vertex
        tour = []
        while current_path:
            current_station = current_path[-1]
            if len(self.get_station_connections(current_station)): # if there are any neighbours of the current station unvisited
                next_vertex = graph[current_station].pop().name # get next unvisited neighbour of current station
                current_path.append(next_vertex) # Push the new vertex to the stack
            else: # back-track to nearest vertex that  has unvisited edges
                tour.append(current_path.pop())
        return tour

    def __str__(self):
        res = ""
        for station_name, connections in self.graph.items():
            res += station_name + ": "
            for station in connections:
                res += str(station) + ", "
            res += "\n"
        return res

class Station:
    def __init__(self, name, latitude, longitude):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.weight = None # the weight of a station is in reference to the station whose dictionary this object find itself in

    def set_weight(self, weight):
        self.weight = weight

    def get_weight(self):
        return self.weight

    def __copy__(self):
        obj = type(self)(self.name, self.latitude, self.longitude)
        obj.set_weight(self.weight)
        return obj

    def __str__(self):
        return str((self.name, self.weight))

# this heap key is made for the A* algorithm
# the custom comparator operators make the code more readable in the minDistance method in LondonRailwayMapper
class HeapKey:
    def __init__(self, name, f_score=None):
        self.name = name
        self.f_score = f_score

    def __lt__(self, x): return self.f_score < x.f_score
    def __le__(self, x): return self.f_score <= x.f_score
    def __eq__(self, x): return self.name == x.name
    def __ne__(self, x): return self.name != x.name
    def __ge__(self, x): return self.f_score >= x.f_score
    def __gt__(self, x): return self.f_score > x.f_score

    def __str__(self): return str((self.name, self.f_score))

# this MSTEdge is made for Hierholzer’s Algorithm
# the custom comparator operators make the code more readable in the get_mst method in Network
class MSTEdge:
    def __init__(self, start_station_name, end_station_name, weight):
        self.start_station_name = start_station_name
        self.end_station_name = end_station_name
        self.weight = weight

    def get_start_station_name(self):
        return self.start_station_name

    def get_end_station_name(self):
        return self.end_station_name

    def get_weight(self):
        return self.weight

    def __lt__(self, x): return self.weight < x.weight
    def __le__(self, x): return self.weight <= x.weight
    def __eq__(self, x): return (self.start_station_name, self.end_station_name) == (x.from_station_name, x.to_station_name)
    def __ne__(self, x): return (self.start_station_name, self.end_station_name) != (x.end_station_name, x.end_station_name)
    def __ge__(self, x): return self.weight >= x.weight
    def __gt__(self, x): return self.weight > x.weight

    def __str__(self):
        return str((self.start_station_name, self.end_station_name, self.weight))

# implementation of a min-heap
class MinHeap:
    def __init__(self):
        self.heap = [None]

    def swap(self, index1, index2):
        temp = self.heap[index1]
        self.heap[index1] = self.heap[index2]
        self.heap[index2] = temp

    # swim the item at index i up to its correct position
    def swim(self, i):
        while i > 1 and self.heap[i//2] > self.heap[i]:
            self.swap(i, i//2)
            i = i//2

    def insert(self, x):
        self.heap.append(x)
        self.swim(self.size())

    # skin the item at index i down to its correct position
    def sink(self, i):
        while 2*i <= self.size():
            j = 2*i
            if j<self.size() and self.heap[j]>self.heap[j + 1]: j+=1
            if self.heap[i] <= self.heap[j]: break
            self.swap(i, j)
            i = j

    def remove_min(self):
        self.heap[1] = self.heap[-1]
        self.heap.pop()
        self.sink(1)

    def get_min(self):
        res = self.heap[1]
        self.remove_min()
        return res

    def contains(self, x):
        return x in self.heap[1:]

    def size(self):
        return len(self.heap)-1

    def is_empty(self):
        return len(self.heap) <= 1

    def __str__(self):
        return str([str(x) for x in self.heap])

Use the cell below to implement the requested API.

In [662]:
import csv
from collections import deque

class LondonRailwayMapper(AbstractLondonRailwayMapper):

    def __init__(self):
        super().__init__()
        self.network = Network()

    # loads stations and lines into self.network
    def loadStationsAndLines(self):

        # loads stations into self.network as unconnected nodes
        with open("londonstations.csv") as londonstations_csv_file:
            csv_reader = csv.reader(londonstations_csv_file, delimiter=',')
            station_count = -1
            for row in csv_reader:
                if station_count != -1:
                    station = Station(row[0], float(row[1]), float(row[2])) # generates station object from each line in "londonstations.csv"
                    self.network.add_station(station)
                station_count += 1

        # adds all edges between stations
        with open("londonrailwaylines.csv") as londonrailwaylines_csv_file:
            csv_reader = csv.reader(londonrailwaylines_csv_file, delimiter=',')
            station_count = -1
            for row in csv_reader:
                if station_count != -1:
                    self.network.add_edge(row[1], row[2]) # adds all edges defined in "londonrailwaylines.csv"
                station_count += 1

    # returns the minimum number of stops that need to be travelled through to get from "fromS" to "toS"
    # implemented using a breadth-first search
    # https://en.wikipedia.org/wiki/Breadth-first_search
    def minStops(self, fromS, toS):
        visited = {station_name: False for station_name in self.network.get_station_names()} # stores whether each station has been visited
        distance = {station_name: 0 for station_name in self.network.get_station_names()} # number of stops between fromS and each station
        visited[fromS] = True # start at the fromS
        queue = deque()
        queue.appendleft(fromS)

        while queue: # while there are still unvisited stations
            current_station_name = queue.pop()
            if current_station_name == toS:
                return distance[toS]
            for station_connection in self.network.get_station_connections(current_station_name):
                # for each unvisited neighbour of the current station,
                # increment its distance from the source node by one and add it to the queue
                if not visited[station_connection.name]:
                    distance[station_connection.name] = distance[current_station_name] + 1
                    visited[station_connection.name] = True
                    queue.appendleft(station_connection.name)

    # returns the minimum distance (in miles) that needs to be travelled to go from "fromS" to "toS"
    # implemented using the A* search algorithm
    # https://en.wikipedia.org/wiki/A*_search_algorithm
    # http://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html
    def minDistance(self, fromS, toS):

        # h is a heuristic function that returns the haversine distance between the given station and toS
        def h(station_name):
            return self.network.distance(station_name, toS)

        start_key_heuristic = h(fromS)

        open_set = MinHeap() # set of currently found stations
        open_set.insert(HeapKey(fromS, start_key_heuristic))

        # g_score is a dictionary containing key value pairs of all station names and the lowest cost path from "fromS" to each station
        g_score = {station_name: math.inf for station_name in self.network.get_station_names()}
        g_score[fromS] = 0

        # f_score is the current estimate for the distance from "fromS" to "toS" going through a given station
        f_score = {station_name: math.inf for station_name in self.network.get_station_names()}
        f_score[fromS] = start_key_heuristic

        while not open_set.is_empty():
            current = open_set.get_min()
            if current.name == toS:
                return round(g_score[current.name], 4)

            for station_connection in self.network.get_station_connections(current.name):
                # tentative_g_score is the distance from "fromS" to the station_connection passing through current
                tentative_g_score = g_score[current.name] + station_connection.get_weight()
                if tentative_g_score < g_score[station_connection.name]: # if going through the current node improves on the g_score then make that the route to it
                    g_score[station_connection.name] = tentative_g_score
                    f_score[station_connection.name] = g_score[station_connection.name] + h(station_connection.name)
                    if not open_set.contains(HeapKey(station_connection.name)):
                        open_set.insert(HeapKey(station_connection.name, f_score[station_connection.name]))

    # returns an ordered list of station names connected pairwise given a list of stations
    # it aims to minimise the total length of track needed to connect all stations
    # it effectively computes a minimal hamiltonian path
    # https://en.wikipedia.org/wiki/Travelling_salesman_problem
    # The Travelling Salesman Problem: A Guided Tour of Combinatorial Optimization (Wiley Series in Discrete Mathematics & Optimization)
    def newRailwayLine(self, inputList):
        railway_line_generator = RailwayLineGenerator(inputList, self.network)
        return railway_line_generator.newRailwayLine()

class RailwayLineGenerator:

    def __init__(self, input_list, original_network):
        self.original_network = original_network # the complete network as loaded from the files
        self.input_list = input_list
        self.network = Network() # a complete weighted graph containing all stations in input_list
        self.generate_graph()

    def generate_graph(self):

        # add stations to graph
        for station_name in self.input_list:
            station_object = self.original_network.get_station(station_name)
            self.network.add_station(station_object)

        # make graph complete
        for station_name in self.input_list:
            for station_connection_name in self.input_list:
                if station_connection_name != station_name:
                    self.network.add_edge(station_name, station_connection_name)

    # a 2 opt algorithm to optimise the solution found using the minimum spanning tree
    # https://en.wikipedia.org/wiki/2-opt
    # https://on-demand.gputechconf.com/gtc/2014/presentations/S4534-high-speed-2-opt-tsp-solver.pdf
    def optimise_solution(self, route):
        n = len(route)
        best = route
        best_cost = self.get_total_distance(best)
        improved = True
        count = 0
        while improved:
            improved = False
            for i in range(1, n-2):
                for j in range(i+1, n):
                    if j-i == 1: continue # no change in route
                    new_route = route[:]
                    new_route[i:j] = route[j-1:i-1:-1] # perform the 2optSwap
                    new_route_cost = self.get_total_distance(new_route) # this is an time expensive call so it is memoized
                    count += 1 # counts the number of get_total_distance calls
                    if new_route_cost < best_cost: # this route is better so store it
                        best = new_route
                        best_cost = new_route_cost
                        improved = True
                    if count > 30000/n*math.log(n, 2): # this limits the time the optimisation can take
                        return best
            route = best
        return best

    # finds the edge in a hamiltonian cycle with the maximum weight and removes it
    # it does this by rotating the pairwise list of stations so the two nodes with the maximum edge weighting appear at either end of the list
    # If 2->3 had the max weighting then remove_max_edge([1, 6, 2, 3, 7, 4]) = [3, 7, 4, 1, 6, 2]
    def remove_max_edge(self, station_list):
        n = len(station_list)
        max_weight = 0
        max_index = 0

        for i in range(n):
            weight = self.network.distance(station_list[i], station_list[(i+1)%n])
            if weight > max_weight:
                max_weight = weight
                max_index = (i+1)%n

        return station_list[max_index:] + station_list[:max_index]

    # returns the total length of track needed for the railway line in the given order
    def get_total_distance(self, station_list):
        total = 0
        for i in range(len(station_list)-1):
            total += self.network.distance(station_list[i], station_list[i+1])
        return total

    # process for creating the new railway line
    # 1. generate a minimum spanning tree
    # 2. find the eulerian tour
    # 3. remove duplicate nodes
    # 4. optimise
    # 5. remove longest edge to form a path
    def newRailwayLine(self):
        mst = self.network.get_mst()
        e_tour = mst.get_eulerian_tour()
        tsp_solution = list(dict.fromkeys(e_tour)) # removes duplicates while preserving order
        tsp_solution = self.optimise_solution(tsp_solution)
        tsp_solution = self.remove_max_edge(tsp_solution)
        return tsp_solution


Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [663]:
import matplotlib.pyplot as plt
import random
import timeit
import numpy as np
londonRailwayMapper = LondonRailwayMapper()
londonRailwayMapper.loadStationsAndLines()

def generate_network(v):
    network = Network()
    station_names = londonRailwayMapper.network.get_station_names()
    count = 0
    while count < v:
        station_name = station_names[random.randint(0, len(station_names)-1)]
        station = londonRailwayMapper.network.get_station(station_name)
        try:
            network.get_station(station.name)
        except KeyError:
            network.add_station(station)
            count+=1
    new_station_names = network.get_station_names()

    edges = 0
    while not network.is_connected():
        edge = random.sample(new_station_names, 2)
        if not network.exists_edge(edge[0], edge[1]):
            network.add_edge(edge[0], edge[1])
            edges += 1

    for i in range(random.randint(20, v*3)):
        edge = random.sample(new_station_names, 2)
        if not network.exists_edge(edge[0], edge[1]):
            network.add_edge(edge[0], edge[1])
            edges += 1

    return network, edges

# experimental analysis of minStops
"""
x = []
y = []

for v in range(30, 653, 10):

    for _ in range(2):
        minStopTester = LondonRailwayMapper()
        minStopTester.network, e = generate_network(v)
        x.append(v+e)
        station_names = minStopTester.network.get_station_names()
        stations = random.sample([i for i in range(v)], 2)
        start_time = timeit.default_timer()
        minStopTester.minStops(station_names[stations[0]], station_names[stations[1]])
        end_time = timeit.default_timer()
        time = end_time-start_time
        y.append(time)
        print(len(x), x, y)

plt.plot(x, y, "o")
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
plt.ylabel("minStops run time (s)")
plt.xlabel("V + E")
plt.savefig('foo.png', dpi=1000, bbox_inches='tight', pad_inches=0.3)
plt.show()
"""

# experimental analysis of minDistance
"""
x = []
y = []

for v in range(30, 653, 10):

    for _ in range(2):
        minDistanceTester = LondonRailwayMapper()
        minDistanceTester.network, e = generate_network(v)
        x.append(v+e)
        station_names = minDistanceTester.network.get_station_names()
        stations = random.sample([i for i in range(v)], 2)
        start_time = timeit.default_timer()
        minDistanceTester.minDistance(station_names[stations[0]], station_names[stations[1]])
        end_time = timeit.default_timer()
        time = end_time-start_time
        y.append(time)
        print(len(x), x, y)

plt.plot(x, y, "o")
plt.ylabel("minDistance run time (s)")
plt.xlabel("V + E")
plt.savefig('foo.png', dpi=1000, bbox_inches='tight', pad_inches=0.3)
plt.show()
"""

# experimental analysis of newRailwayLine
"""
x = []
y = []
for n in range(5, 300, 5):
    for _ in range(5):
        station_list = random.sample(londonRailwayMapper.network.get_station_names(), n)
        start_time = timeit.default_timer()
        londonRailwayMapper.newRailwayLine(station_list)
        end_time = timeit.default_timer()
        time = end_time-start_time
        x.append(n)
        y.append(time)
        print(len(x), x, y)

plt.plot(x, y, "o")
plt.ylabel("newRailwayLine run time (s)")
plt.xlabel("N")
plt.savefig('foo.png', dpi=1000, bbox_inches='tight', pad_inches=0.3)
plt.show()
"""

'\nx = []\ny = []\nfor n in range(5, 300, 5):\n    for _ in range(5):\n        station_list = random.sample(londonRailwayMapper.network.get_station_names(), n)\n        start_time = timeit.default_timer()\n        londonRailwayMapper.newRailwayLine(station_list)\n        end_time = timeit.default_timer()\n        time = end_time-start_time\n        x.append(n)\n        y.append(time)\n        print(len(x), x, y)\n\nplt.plot(x, y, "o")\nplt.ylabel("newRailwayLine run time (s)")\nplt.xlabel("N")\nplt.savefig(\'foo.png\', dpi=1000, bbox_inches=\'tight\', pad_inches=0.3)\nplt.show()\n'

The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [664]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")

#
# testing the newRailwayLine() API on a small list of stations
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.016

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Baker Street to North Wembley in 6 stops and 6 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Epping to Belsize Park in 17 stops and 17 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Canonbury to Balham in 10 stops and 10 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Vauxhall to Leytonstone in 6 stops and 6 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Queens Park', 'Swiss Cottage', 'Highgate', 'Moorgate', 'Liverpool Street', 'Chigwell']
Total track length from Queens Park to Chigwell : 18.697 miles
Execution time newLine: 0.001


Station list ['Abbey Road', 'Barbican', 'Bethnal Green', 'Cambridge Heath', 'Covent Garden', 'Dollis Hill', 'East Finchley', 'Finchley Road and Frognal', 'Great Portland Street', 'Hackney Wick', 'Islewor